#Importing datasets into pandas

In [27]:
import matplotlib as plt
import kaggle
import zipfile
import pandas as pd

In [28]:
us_pollution_filename = 'pollution_us_2000_2016.csv'
kaggle.api.dataset_download_file('sogun3/uspollution', us_pollution_filename)
zip_ref = zipfile.ZipFile(us_pollution_filename+'.zip', 'r')
zip_ref.extractall()
zip_ref.close()
df_us_pollution = pd.read_csv(us_pollution_filename)

In [30]:
resp_data_path = 'IHME_USA_COUNTY_RESP_DISEASE_MORTALITY_1980_2014_NATIONAL_Y2017M09D26.XLSX'
df_resp_disease = pd.read_excel(resp_data_path)

In [31]:
df_resp_disease

,Chronic respiratory diseases,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Location,FIPS,"Mortality Rate, 1980*","Mortality Rate, 1985*","Mortality Rate, 1990*","Mortality Rate, 1995*","Mortality Rate, 2000*","Mortality Rate, 2005*","Mortality Rate, 2010*","Mortality Rate, 2014*","% Change in Mortality Rate, 1980-2014"
1,United States,NaN,"40.79 (39.80, 41.77)","42.99 (42.10, 43.90)","44.94 (44.08, 45.93)","50.07 (48.98, 51.23)","54.56 (53.34, 55.74)","54.58 (53.39, 55.85)","52.92 (51.72, 54.16)","52.92 (51.60, 54.36)","29.73 (25.47, 33.83)"
2,Alabama,1,"42.43 (41.22, 43.60)","46.29 (45.24, 47.34)","49.95 (48.86, 51.12)","57.00 (55.78, 58.36)","65.65 (64.23, 67.09)","69.21 (67.66, 70.75)","70.86 (69.27, 72.46)","73.44 (71.54, 75.49)","73.08 (67.09, 79.45)"
3,"Autauga County, Alabama",1001,"46.56 (42.86, 50.68)","50.68 (47.16, 54.42)","55.55 (52.19, 59.07)","63.21 (59.85, 66.62)","71.77 (68.27, 75.56)","77.13 (73.44, 81.17)","77.75 (73.63, 81.85)","81.79 (77.37, 86.77)","75.67 (58.34, 95.40)"
4,"Baldwin County, Alabama",1003,"37.15 (34.89, 39.40)","39.46 (37.38, 41.68)","42.97 (40.88, 45.19)","48.15 (46.05, 50.62)","52.07 (49.85, 54.48)","52.56 (50.47, 54.77)","55.12 (52.87, 57.56)","54.28 (51.92, 56.65)","46.10 (35.48, 59.57)"
5,"Barbour County, Alabama",1005,"42.84 (39.76, 46.26)","46.78 (43.78, 50.05)","50.05 (47.18, 53.20)","57.07 (54.07, 60.08)","65.19 (61.87, 68.69)","66.93 (63.11, 70.52)","67.52 (63.26, 71.55)","69.82 (65.35, 74.56)","62.98 (47.20, 80.68)"
6,"Bibb County, Alabama",1007,"46.14 (42.19, 50.25)","51.21 (47.49, 55.26)","56.24 (52.68, 59.96)","65.08 (61.25, 69.46)","74.13 (69.86, 78.72)","76.93 (72.79, 81.34)","80.51 (75.82, 85.85)","84.49 (78.58, 90.47)","83.11 (62.21, 105.79)"
7,"Blount County, Alabama",1009,"43.68 (40.39, 46.99)","47.93 (44.95, 51.00)","52.70 (49.58, 55.60)","62.08 (58.87, 65.25)","71.95 (68.40, 75.50)","77.68 (73.99, 81.36)","80.48 (76.67, 84.46)","87.05 (82.51, 91.87)","99.30 (80.27, 120.26)"
8,"Bullock County, Alabama",1011,"35.56 (31.98, 39.38)","37.86 (34.52, 41.19)","39.72 (36.37, 43.00)","45.46 (41.83, 49.14)","50.81 (46.83, 55.06)","53.41 (48.87, 58.10)","53.25 (48.76, 58.40)","51.49 (46.55, 56.67)","44.81 (27.81, 63.90)"
9,"Butler County, Alabama",1013,"33.28 (30.46, 36.21)","37.34 (34.69, 40.17)","42.14 (39.22, 45.27)","51.16 (48.07, 54.41)","60.60 (57.06, 64.18)","68.40 (64.12, 72.48)","70.43 (66.01, 74.90)","75.09 (69.58, 80.45)","125.60 (100.34, 154.14)"


In [33]:
df_us_pollution

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.200,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.200,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.200,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.200,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.600,23,NaN
5,5,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,1.066667,2.300,0,26.0
6,6,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.937500,2.6,23,NaN,Parts per million,0.850000,1.600,23,NaN
7,7,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.937500,2.6,23,NaN,Parts per million,1.066667,2.300,0,26.0
8,8,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-03,Parts per billion,...,Parts per billion,5.250000,11.0,19,16.0,Parts per million,1.929167,4.400,8,NaN
9,9,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-03,Parts per billion,...,Parts per billion,5.250000,11.0,19,16.0,Parts per million,1.762500,2.500,8,28.0
